In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with Cloud Deploy Vertex AI Model Deployer

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_registry/get_started_with_vertex_ai_deployer.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_registry/get_started_with_vertex_ai_deployer.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_registry/get_started_with_vertex_ai_deployer.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

After a model is trained, validated and added to model registry, it is ready for deployment. Before deploying the model, you may need to  go through a continuous deployment process deployment process to test and validate the model.

This tutorial demonstrates how to deploy a Vertex AI model to an endpoint using a Cloud Deploy custom target, which allows you to specify a Vertex AI Endpoint as runtime environment into which to deploy your model.

Learn more about [Cloud Deploy](https://cloud.google.com/deploy) and [Vertex AI](https://cloud.google.com/vertex-ai)

### Objective

In this tutorial, you learn how to deploy a Vertex AI Model to an endpoint usign a Cloud Deploy custom target:

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Model Registry
- Vertex AI Endpoint
- Cloud Deploy

The steps performed include:

- Import a Vertex AI model into model registry and create an endpoint where the model will be deployed.
- Define a Cloud Deploy delivery pipeline, custom target type for Vertex AI, and one target.
- Create a Cloud Deploy release and rollout to deploy a Vertex AI model to the target.

### Dataset

The dataset used for the pre-trained model is the Boston Housing dataset contains information collected by the U.S Census Service concerning housing in the area of Boston Mass.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Cloud Deploy

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and [Cloud Deploy pricing](https://cloud.google.com/cloud-deploy/pricing)
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

<div class="alert alert-block alert-warning">
<b>⚠️ This is not an officially supported Google product, and it is not covered by a Google Cloud support contract.
To report bugs or request features in a Google Cloud product, please contact Google Cloud support⚠️</b>
</div>

## Installation

Install the following packages required to execute this notebook.

{TODO: Suggest using the latest major GA version of each package; i.e., --upgrade}

In [ ]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""
! pip3 install {USER} --upgrade google-cloud-aiplatform

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable APIs](https://console.cloud.google.com/flows/enableapi?apiid=clouddeploy.googleapis.com,containeranalysis.googleapis.com,compute.googleapis.com,aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### Set permissions for the service account

**Important**: Before to run the following cell, if you are authenticated with the service account, check that it has the required permissions. See the `Before to begin` section [here](https://cloud.google.com/iam/docs/manage-access-service-accounts#before_you_begin) for more details.

To use Cloud Deploy, you need to set the following roles on your service account:

* `roles/clouddeploy.jobRunner` this is required to create cloud deploy resources, and to create releases and deployments against a pipeline.

* `roles/clouddeploy.developer` required to access cloud deploy resources.

*  `roles/clouddeploy.operator` required to manage Cloud Deploy pipelines, target resources, releases, rollouts, and job runs.

*  `roles/clouddeploy.customTargetTypeAdmin` required to have full control of Cloud Deploy custom target types.

* `roles/containeranalysis.notes.editor` required for the custom image to run Artifact Analysis along the release process

* `roles/aiplatform.user` required for the custom image to create and import models and to access model information, and to make model deployments.


In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.jobRunner"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.developer"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.operator"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.customTargetTypeAdmin"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/containeranalysis.notes.editor"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/aiplatform.user"

### Import libraries

In [ ]:
from google.cloud import aiplatform as vertex_ai

### Set variables

Below you set the model and the serving image you use in this tutorial.

In [ ]:
MODEL_URI = "gs://cloud-samples-data/vertex-ai/model-deployment/models/boston/model"
DEPLOY_IMAGE = (
    "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-cpu.nightly:latest"
)

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Introduction to Cloud Deploy Vertex AI Deployer

Cloud Deploy Vertex AI Deployer is a custom target for Cloud Deploy.

Cloud Deploy is a managed continuous delivery service. Users can use Cloud Deploy to define a delivery pipeline and configure an ordered sequence of targets. With custom targets, users can deploy to other systems besides the supported runtimes including Vertex AI endpoints.

With Vertex AI, once the Vertex AI target endpoint is defined, users can create a release, which is associated with a specific version of the model. Then they can create a rollout, which is a deployment of that a model release to a specific target endpoint in the pipeline sequence. And finally, users can promote this release to the next target in the sequence.


### Register a model into Vertex AI Model Registry

You upload your model as a version of the `Model` resource in the `Model Registry`. At minimum, you specify the display name, the Cloud Bucket location of the model and the Prediction Image you want to use.

For this tutorial, you use a pre-trained model from Vertex AI's cloud samples bucket. For the docker image, you use a Cloud AI optimized version of tensorflow: [tf_opt](https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime).

In [ ]:
registered_model = vertex_ai.Model.upload(
    display_name="model_to_deploy",
    artifact_uri=MODEL_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(registered_model)

### Create a Vertex AI Endpoint

You create an `Endpoint` resource using the `Endpoint.create()` method. At a minimum, you specify the display name for the endpoint.

In [ ]:
endpoint = vertex_ai.Endpoint.create(
    display_name="target_endpoint",
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint)

### Create delivery pipeline, target, and custom target type

When you use Vertex AI Deployer to deploy a registered model, you cover the following steps:

- You define a custom action which is similar to deploy hooks and is defined in the skaffold.yaml file.

- You define a custom target type which is a Cloud Deploy resource identifying the custom action used by targets of this type. In your case, a Vertex AI Endpoint.

- You set a target definition for a custom target is the same as for any target type, except that it includes some properties.

Once you have all the necessary components, you setup a Cloud Deploy delivery pipeline that references the configured target. And you can fully utilize Cloud Deploy features like promotion, approvals, and rollbacks by referencing the target to deploy you model. If you want to know more about custom targets and how custom targets work in Cloud Deploy, check out the [official documentation](https://cloud.google.com/deploy/docs/custom-targets).

To cover all these steps, you can use the `build_and_register.sh` script in the [vertex-ai](https://github.com/GoogleCloudPlatform/cloud-deploy-samples/tree/main/custom-targets/vertex-ai) directory. The script can be used to build the Vertex AI model deployer image and register a Cloud Deploy custom target type that references the image.

Below, you clone the Cloud Deploy samples repository and set the current directory to the vertex AI sample quickstart folder.

In [ ]:
!git clone https://github.com/googlecloudplatform/cloud-deploy-samples.git
%cd cloud-deploy-samples/custom-targets/vertex-ai/quickstart

Run the script that builds the image and registers the custom target type.

In [ ]:
!../build_and_register.sh -p $PROJECT_ID -r $REGION

Run the following command to fill in the placeholders in the Cloud Deploy and Skaffold configuration values with the actual images.


In [ ]:
endpoint_id = endpoint.name

!./replace_variables.sh -p $PROJECT_ID -r $REGION -e $endpoint_id

Finally, apply the Cloud Deploy configuration defined in `clouddeploy.yaml`.

In [ ]:
! gcloud deploy apply --file=clouddeploy.yaml --project=$PROJECT_ID --region=$REGION

### Create a release and rollout

Create a Cloud Deploy release for the `configuration` defined in the configuration directory. Notice that the configuration file contains deployment settings including machine type, number of replicas and more which are associated with your endpoint. This automatically creates a rollout that deploys the first model version to the target endpoint.

To create a Cloud Deploy release, you have to specify

* The `--source` command line flag which instructs gcloud where to look for the configuration files relative to the working directory where the command is run.

* The `--deploy-parameters` flag which is used to provide the custom deployer with additional parameters needed to perform the deployment.

Here, you provide the custom deployer with two parameters:

* `customTarget/vertexAIModel` which indicates the full resource name of the model to deploy

* `--delivery-pipeline` which is the name of the delivery pipeline where the release will be created, and the project and region of the pipeline
is specified by `--project` and `--region` respectively.

In [ ]:
model_id = registered_model.name
deploy_params = f'customTarget/vertexAIModel=projects/{PROJECT_ID}/locations/{REGION}/models/{model_id}'

! gcloud deploy releases create release-0001 \
    --delivery-pipeline=vertex-ai-cloud-deploy-pipeline \
    --project=$PROJECT_ID \
    --region=$REGION \
    --source=configuration \
    --deploy-parameters=$deploy_params

### Monitor the release's progress

To check release details, run the command below.

In [ ]:
!gcloud deploy releases describe release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION

Run this command to filter only the render status of the release.


In [ ]:
!gcloud deploy releases describe release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION --format "(renderState)"

<div class="alert alert-block alert-warning">
<b>
⚠️ It will take up to 15 minutes for the model to fully deploy. ⚠️
</b>
</div>

### Monitor rollout status

In the [Cloud Deploy UI](https://cloud.google.com/deploy) for your project click on the
`vertex-ai-cloud-deploy-pipeline` delivery pipeline. Here you can see the release created and the rollout to the dev target for the release.

You can also describe the rollout created using the following command.

In [ ]:
! gcloud deploy rollouts describe release-0001-to-prod-endpoint-0001 --release=release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION

### Inspect Endpoint of the deployed model

After the rollout completes, you can inspect the deployed models and traffic splits of the endpoint with `gcloud`.

In [ ]:
! gcloud ai endpoints describe $endpoint_id --region $REGION --project $PROJECT_ID

### Inspect aliases in the deployed model

Monitor the post-deploy operation by querying the rollout.


In [ ]:
!gcloud deploy rollouts describe release-0001-to-prod-endpoint-0001 --release=release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION --format "(phases[0].deploymentJobs.postdeployJob)"

After the post-deploy job has succeeded, you can then inspect the deployed model and view its currently assigned aliases. `prod` and `champion` should be assigned. Those aliases would help you to manage the delivery process of your model.


In [ ]:
! gcloud ai models describe $model_id --region $REGION --project $PROJECT_ID --format "(versionAliases)"

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial as shown below.

In [ ]:
delete_endpoint = True
delete_model = True
delete_cloud_deploy = True

if delete_endpoint or os.getenv("IS_TESTING"):
    endpoint_list = vertex_ai.Endpoint.list(filter='display_name="target_endpoint"')
    for endpoint in endpoint_list:
        endpoint.delete(force=True)

if delete_model or os.getenv("IS_TESTING"):
    model_list = vertex_ai.Model.list(filter='display_name="model_to_deploy"')
    for model in model_list:
        model.delete()

if delete_model or os.getenv("IS_TESTING"):
    ! gcloud deploy delete --file=clouddeploy.yaml --force --project=$PROJECT_ID --region=$REGION